In [14]:
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
#import rpy2.robjects as ro
pandas2ri.activate()
#ro.r('''.libPaths('C:/Users/aduprey/Documents/R/win-library/3.5')''')
rblpapi = importr('Rblpapi')
base = importr('base')
stats = importr("stats")
utils = importr('utils')

In [15]:
import bbgclient
import dfutils
import datetime
import pandas as pd
import json
import numpy as np
import statsmodels.formula.api as sm

In [16]:
#alpha = "JACK US EQUITY"
#unaffected_date = "2017-05-16"
#tgt_date = "2018-08-08"
lookback = 120 #120 days (6 months) - is this the proper duration?

#analyst_upside = 62.5
multiples_names = ["EV/Sales", "EV/EBITDA", "P/E", "DVD Yield", "FCF Yield"]
#multiples_weights = [0,0,1,0,0]
#multiplesVSweights = dict(zip(multiples_names, multiples_weights))

#The functions as for the api_host and 
api_host = bbgclient.bbgclient.get_next_available_host()

In [17]:
def multiple_underlying_df(ticker, start_date_yyyymmdd, end_date_yyyymmdd, api_host, fperiod):
    
    def last_elem_or_null2(secid2mnemonic):
        try:
            secid = secid2mnemonic.keys()[0]
            mnemonic2value = secid2mnemonic[secid]
            mnemonic = mnemonic2value.keys()[0]
            values = mnemonic2value[mnemonic]
            return float(values[-1])
        except:
            return None
    def last_elem_or_null(ts):
        if ts is None: return None
        if len(ts) == 0: return None
        return ts.iloc[-1]
    #13 mnemonics

    #px = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['PX_LAST'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #mkt_cap = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['CUR_MKT_CAP'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #ev_component = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['CUR_EV_COMPONENT'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #eqy_sh_out = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['EQY_SH_OUT'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #best_ebitda = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_EBITDA'],start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod}, api_host=api_host))
    #best_sales = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_SALES'],start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod}, api_host=api_host))
    #best_eps = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_EPS'],start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod}, api_host=api_host))
    #div_ind_yield = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['DIVIDEND_INDICATED_YIELD'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #best_opp = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_OPP'],start_date_yyyymmdd,end_date_yyyymmdd, api_host=api_host))
    #best_ni = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_NET_INCOME'],start_date_yyyymmdd,end_date_yyyymmdd,overrides_dict={'BEST_FPERIOD_OVERRIDE':fperiod},api_host=api_host))
    #best_capex = last_elem_or_null2(bbgclient.bbgclient.get_secid2field([ticker],'tickers',['BEST_CAPEX'],start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))

    px = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'PX_LAST',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    mkt_cap = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'CUR_MKT_CAP',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    ev_component = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'CUR_EV_COMPONENT',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    eqy_sh_out = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'EQY_SH_OUT',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_ebitda = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_EBITDA',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    best_sales = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_SALES',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    best_eps = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_EPS',start_date_yyyymmdd,end_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host))
    div_ind_yield = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'DIVIDEND_INDICATED_YIELD',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_opp = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_OPP',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))
    best_ni = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_NET_INCOME',start_date_yyyymmdd,end_date_yyyymmdd, {'BEST_FPERIOD_OVERRIDE':fperiod}, api_host=api_host))
    best_capex = last_elem_or_null(bbgclient.bbgclient.get_timeseries(ticker,'BEST_CAPEX',start_date_yyyymmdd,end_date_yyyymmdd,api_host=api_host))

    cols = ['Date','PX','CUR_MKT_CAP','EQY_SH_OUT','BEST_EBITDA','BEST_SALES',
            'BEST_EPS','DIVIDEND_INDICATED_YIELD','BEST_OPP','BEST_NET_INCOME','BEST_CAPEX','CUR_EV_COMPONENT']
    datum = [(pd.to_datetime(end_date_yyyymmdd),px,mkt_cap,eqy_sh_out,best_ebitda,best_sales,best_eps,
                                          div_ind_yield,best_opp,best_ni,best_capex,ev_component)]
    df = pd.DataFrame(columns = cols,data = datum)

    return df

In [18]:
def multiples_df(ticker, start_date_yyyymmdd, unaffected_date_yyyymmdd, api_host, fperiod, multiples_to_query='ALL'):
    if multiples_to_query == 'ALL':
        multiples_to_query = ['EV/EBITDA','EV/Sales','P/E','DVD yield','FCF yield']
    pe = pd.Series()
    ev_to_ebitda = pd.Series()
    ev_to_sales = pd.Series()
    dvd_yield = pd.Series()
    px = bbgclient.bbgclient.get_timeseries(ticker,'PX_LAST',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host)
    if 'EV/EBITDA' in multiples_to_query:
        ev_to_ebitda = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CUR_EV_TO_EBITDA',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'EV/Sales' in multiples_to_query:
        ev_to_sales = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CURRENT_EV_BEST_SALES',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'P/E' in multiples_to_query:
        pe = bbgclient.bbgclient.get_timeseries(ticker,'BEST_PE_RATIO',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host)
    if 'DVD yield' in multiples_to_query:
        dvd_yield = bbgclient.bbgclient.get_timeseries(ticker,'DIVIDEND_INDICATED_YIELD',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host)

    df = px.reset_index().rename(columns={'index':'Date',0:'PX'})
    if 'FCF yield' in multiples_to_query:
        #### FCF-related mnemonics
        ebitda = bbgclient.bbgclient.get_timeseries(ticker,'BEST_EBITDA',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'EBITDA'})
        opp = bbgclient.bbgclient.get_timeseries(ticker,'BEST_OPP',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'OPP'})
        capex = bbgclient.bbgclient.get_timeseries(ticker,'BEST_CAPEX',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'CAPEX'})
        eqy_sh_out = bbgclient.bbgclient.get_timeseries(ticker,'EQY_SH_OUT',start_date_yyyymmdd,unaffected_date_yyyymmdd,api_host=api_host).reset_index().rename(columns={'index':'Date',0:'EQY_SH_OUT'})
        ni = bbgclient.bbgclient.get_timeseries(ticker,'BEST_NET_INCOME',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'NI'})
        #eps = bbgclient.bbgclient.get_timeseries(ticker,'BEST_EPS',start_date_yyyymmdd,unaffected_date_yyyymmdd,{'BEST_FPERIOD_OVERRIDE':fperiod},api_host).reset_index().rename(columns={'index':'Date',0:'EPS'})
        ####
        fcf = pd.merge(px.reset_index().rename(columns={'index':'Date',0:'PX'}),ebitda, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,opp, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,capex, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,eqy_sh_out, how='left', on=['Date']).ffill().bfill()
        fcf = pd.merge(fcf,ni, how='left', on=['Date']).ffill().bfill()
        #fcf = pd.merge(fcf,eps, how='left', on=['Date']).ffill().bfill()
        #fcf['FCF'] = (fcf['NI'] + fcf['EBITDA'] - fcf['OPP'] + fcf['CAPEX'])/(fcf['NI']/fcf['EPS'])
        fcf['FCF'] = (fcf['NI'] + fcf['EBITDA'] - fcf['OPP'] + fcf['CAPEX'])/fcf['EQY_SH_OUT']
        fcf['FCF yield'] = fcf['FCF']/fcf['PX']
        df = pd.merge(df, fcf[['Date','FCF yield']], how='left',on='Date').ffill().bfill()

    df = pd.merge(df,pe.reset_index().rename(columns={'index':'Date',0:'P/E'}), how='left', on=['Date']).ffill().bfill()
    df = pd.merge(df, ev_to_ebitda.reset_index().rename(columns={'index':'Date',0:'EV/EBITDA'}), how='left',on='Date').ffill().bfill()
    df = pd.merge(df, ev_to_sales.reset_index().rename(columns={'index':'Date',0:'EV/Sales'}), how='left',on='Date').ffill().bfill()
    df = pd.merge(df, dvd_yield.reset_index().rename(columns={'index':'Date',0:'DVD yield'}), how='left',on='Date').ffill().bfill()

    # Date     PX     P/E  EV/EBITDA  EV/Sales  DVD yield  FCF yield
    return df

In [19]:
def compute_implied_price_from_multiple(metric_name, multiple, mult_underlying_df):
    try:
        if metric_name == 'EV/EBITDA':
            ebitda = float(mult_underlying_df['BEST_EBITDA'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            ev_component = float(mult_underlying_df['CUR_EV_COMPONENT'].iloc[0])
            # net_debt =  float(mult_underlying_df['BEST_NET_DEBT'][0])
            # minority = float(mult_underlying_df['MINORITY_NONCONTROLLING_INTEREST'][0])
            # pfd_eqy = float(mult_underlying_df['BS_PFD_EQY'][0])
            return ((multiple*ebitda)-ev_component)/eqy_sh_out

        if metric_name == 'EV/Sales':
            sales = float(mult_underlying_df['BEST_SALES'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            ev_component = float(mult_underlying_df['CUR_EV_COMPONENT'].iloc[0])
            #net_debt =  float(mult_underlying_df['BEST_NET_DEBT'].iloc[0])
            #pfd_eqy = float(mult_underlying_df['BS_PFD_EQY'].iloc[0])
            #minority = float(mult_underlying_df['MINORITY_NONCONTROLLING_INTEREST'].iloc[0])
            return ((multiple*sales)-ev_component)/eqy_sh_out

        if metric_name == 'P/E':
            eps = float(mult_underlying_df['BEST_EPS'].iloc[0])
            return eps*multiple

        if metric_name == 'DVD yield':
            curr_dvd_yield = float(mult_underlying_df['DIVIDEND_INDICATED_YIELD'].iloc[0])
            curr_px = float(mult_underlying_df['PX'].iloc[0])
            curr_dvd = curr_dvd_yield*curr_px
            implied_px = curr_dvd/multiple
            return implied_px

        if metric_name == 'FCF yield':
            ni = float(mult_underlying_df['BEST_NET_INCOME'].iloc[0])
            ebitda = float(mult_underlying_df['BEST_EBITDA'].iloc[0])
            opp = float(mult_underlying_df['BEST_OPP'].iloc[0])
            capex = float(mult_underlying_df['BEST_CAPEX'].iloc[0])
            #eps = float(mult_underlying_df['BEST_EPS'].iloc[0])
            eqy_sh_out = float(mult_underlying_df['EQY_SH_OUT'].iloc[0])
            fcf = (ni + ebitda - opp + capex)/eqy_sh_out
            # fcf = (ni + ebitda - opp + capex)/(ni/eps)
            implied_px = fcf/multiple
            return implied_px

    except Exception as e:
        print('failed calculating implied price from multiple: ' + str(metric_name) + ' ' + str(e.args))
        #dbutils.Wic.log('ESS PREMIUM ANALYSIS','failed calculating implied price from multiple: ' + str(e.message))
        return None
    

In [20]:
def bloomberg_peers_from_r(alpha):
    con = rblpapi.blpConnect(host = base.getOption("blpHost", '10.16.1.16'), port = base.getOption("blpPort", '8194'), default = True)
    peers = rblpapi.bds(alpha, "Bloomberg Peers", con = con)
    peers_list = list(peers[0])
    #concatenate "EQUITY" to the end of every peer
    for i in range(0,len(peers_list)):
        peers_list[i] = peers_list[i] + " EQUITY"
    return(peers_list)
    

In [21]:
def premium_analysis_df_OLS(alpha_ticker, analyst_upside, lookback_period, unaffect_dt, tgt_dt, metrics, api_host, fperiod = "1BF"):
    slicer = dfutils.df_slicer()
    unaff_dt = datetime.datetime.strptime(unaffect_dt, '%Y-%m-%d')
    price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    peer_ticker_list = bloomberg_peers_from_r(alpha_ticker)
    
    alpha_historical_mult_df = multiples_df(alpha_ticker, slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod) # ['CID','Date','Ticker','PX','P/E','EV/EBITDA','EV/Sales','DVD yield','FCF yield']
    peer2historical_mult_df = {p:multiples_df(p,slicer.prev_n_business_days(lookback_period,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod, multiples_to_query=metrics) for p in peer_ticker_list}
    ticker2short_ticker = {p:p.split(' ')[0] for p in peer_ticker_list+[alpha_ticker]}
    
    peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
    peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
    alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
    
    #Runs the regression on all metrics and find the optimal peer group per metric which it then uses to calculate the downside
    rows=[]
    metric2peer2coeff = {m:{} for m in metrics}
    peers_d = {}
    for metric in metrics:
        if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
            m_df = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
            for p in peer2historical_mult_df:
                m_df = pd.merge(m_df,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
            peer_tickers = [p for p in peer_ticker_list if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
            peer_tickers = [p for p in peer_tickers if ~peer2ptd_multiple[p][metric].isnull().all() and ~peer2now_multiple[p][metric].isnull().all()]
            m_ols_df = m_df[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in peer_tickers]]
            formula = alpha_ticker.split(' ')[0] + '~.' #+ " + ".join([t.split(' ')[0] for t in peer_ticker_list])
            r_df = ro.pandas2ri.py2ri(m_ols_df)
            r_df = stats.na_omit(r_df)
            model = stats.lm(formula, data = r_df)
            optimal_model = stats.step(model, direction = "backward", trace = False)
            summary_results = base.summary(optimal_model)
            summary_results2 = ro.pandas2ri.ri2py(summary_results)
            optimal_peers = base.attr(optimal_model.rx2('terms'), "term.labels")
            optimal_peers2 = []
            for i in optimal_peers:
                for k in peer_tickers:
                    if k.startswith(i + ' '):
                        optimal_peers2.append(k)
                    elif i.startswith('X') and k.startswith(i[1:] + ' '):
                        optimal_peers2.append(k)
                    elif '.' in i:
                        i = i.replace('.', '/')
                        if k.startswith(i + ' '):
                            optimal_peers2.append(k)
                    #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
            peers_d[metric] = optimal_peers2
            optimal_model_coeff = optimal_model.rx2('coefficients')
            optimal_model_coeff_df = pd.DataFrame(columns=['Peer','Coefficient'],data=[(optimal_peers2[count],optimal_model_coeff[count + 1]) for count in range(0,len(optimal_model_coeff.names)-1)])
            optimal_model_coeff_df = optimal_model_coeff_df.append({'Peer': 'Intercept', 'Coefficient': optimal_model_coeff[0]}, ignore_index = True)
            peer2coeff = {p:optimal_model_coeff.rx2(p)[0] for p in optimal_model_coeff.names}
            del peer2coeff["(Intercept)"]
            count = 0
            for k in optimal_peers:
                peer2coeff[optimal_peers2[count]] = peer2coeff.pop(k)
                count = count + 1
            peer2coeff["(Intercept)"] = optimal_model_coeff[0]
            metric2peer2coeff[metric] = peer2coeff
            rows.append([metric, summary_results2, optimal_model_coeff_df]) #+[peer2coeff[p] for p in optimal_peers2]+[peer2coeff['(Intercept)']])
        else:
            continue
    
    
    df = pd.DataFrame(columns=['Metric', 'Summary Results', 'Coefficients'] ,data=rows)
    df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+metric2peer2coeff[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
    df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

    df['Peers Multiples DataFrame @ Now'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
    df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+metric2peer2coeff[m]['(Intercept)'])
    df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
    df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]

    df['Alpha Upside (analyst)'] = analyst_upside
    df['Premium'] = (df['Alpha Upside (analyst)'].astype(float)-df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

    df['Alpha Downside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)
    df['Alpha Upside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)+df['Premium'].astype(float)
    
    peer_set = {k:set(v) for k, v in peers_d.items()}
    overlap_peers = set.intersection(*peer_set.values()) #Select overlap group from the optimal regression peers above
    
    #Regression of the overlap group which is then used to calculate downside based on all metrics
    rows2=[]
    metric2peer2coeff2 = {m:{} for m in metrics}
    for metric in metrics:
        if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
            m_df2 = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
            for p in overlap_peers:
                m_df2 = pd.merge(m_df2,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
            overlap_peers = [p for p in overlap_peers if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
            m_ols_df2= m_df2[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in overlap_peers]]
               #regress a vs. p1,p2,...,pn
            formula = alpha_ticker.split(' ')[0] + ' ~. ' #+ " + ".join([t.split(' ')[0] for t in overlap_peers])
            r_df2 = ro.pandas2ri.py2ri(m_ols_df2)
            r_df2 = stats.na_omit(r_df2)
            model2 = stats.lm(formula, data = r_df2)
            ols_result = sm.ols(formula=formula, data=m_ols_df).fit()
            summary = base.summary(model2)
            summary = ro.pandas2ri.ri2py(summary)
            model_coeff = model2.rx2('coefficients')
            model_terms = base.attr(model2.rx2('terms'), "term.labels")
            overlap_peers2 = []
            for i in model_terms:
                for k in overlap_peers:
                    if k.startswith(i + ' '):
                        overlap_peers2.append(k)
                    elif i.startswith('X') and k.startswith(i[1:] + ' '):
                        overlap_peers2.append(k)
                    elif '.' in i:
                        i = i.replace('.', '/')
                        if k.startswith(i + ' '):
                            overlap_peers2.append(k)
                    #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
            peer2coeff2 = {}
            for p in range(0,len(overlap_peers)):
                peer2coeff2[overlap_peers2[p]] = model_coeff[p+1]
            peer2coeff2['Intercept'] = model_coeff[0]
            metric2peer2coeff2[metric] = peer2coeff2
            rows2.append([metric,summary]+[peer2coeff2[p] for p in overlap_peers2]+[peer2coeff2['Intercept']])
        else:
            continue

    df2 = pd.DataFrame(columns=['Metric','OLS HTML'] + overlap_peers2 + ['Intercept'], data=rows2)
    df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers2]))
    df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([metric2peer2coeff2[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers2])+metric2peer2coeff2[m]['Intercept'])
    df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
    df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]

    df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers2]))
    df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([metric2peer2coeff2[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers2])+metric2peer2coeff2[m]['Intercept'])
    df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
    df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]

    df2['Alpha Upside (analyst)'] = analyst_upside
    df2['Premium'] = (df2['Alpha Upside (analyst)'].astype(float)-df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

    df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)
    df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)+df2['Premium'].astype(float)

    return {
        #'alpha_historical_mult_df':alpha_historical_mult_df,
        #'peer2historical_mult_df':peer2historical_mult_df,
        'overlap_peers':overlap_peers,
        'df_1': df,
        'df_2': df2
    }

In [22]:
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD yield', 'FCF yield']
OLS_results = premium_analysis_df_OLS('WSC US EQUITY', 15.5, 120, '2018-06-19', "2019-01-18", metrics, api_host, fperiod = "1BF")

SyntaxError: invalid syntax (<unknown>, line 1)

In [203]:
OLS_results['df_1']

,Metric,OLS HTML,JW/A US EQUITY,GCI US EQUITY,GTN US EQUITY,Intercept,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Peers Multiples DataFrame @ Now,Alpha Implied Multiple @ Now,Alpha Balance Sheet DataFrame @ Now,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Premium,Alpha Downside (Adj),Alpha Upside (Adj)
0,P/E,"[[function (formula, data, subset, weights, na...",1.599068,0.119672,0.922674,-22.124512,Peer Multiple 0 JW/A US EQUITY ...,18.541429,Date PX CUR_MKT_CAP EQY_SH_OUT ...,2.873921,Peer Multiple 0 JW/A US EQUITY ...,17.856519,Date PX CUR_MKT_CAP EQY_SH_OUT ...,2.785617,17.87,14.996079,2.785617,17.781695
1,EV/EBITDA,"[[function (formula, data, subset, weights, na...",-1.456129,-1.064005,-3.872278,52.695304,Peer Multiple 0 JW/A US EQUITY ...,2.820167,Date PX CUR_MKT_CAP EQY_SH_OUT ...,9.353480,Peer Multiple 0 JW/A US EQUITY ...,3.522320,Date PX CUR_MKT_CAP EQY_SH_OUT ...,11.324185,17.87,8.516520,11.324185,19.840704
2,EV/Sales,"[[function (formula, data, subset, weights, na...",0.030196,0.840714,-0.716374,1.964105,Peer Multiple 0 JW/A US EQUITY ...,0.361273,Date PX CUR_MKT_CAP EQY_SH_OUT ...,11.487457,Peer Multiple 0 JW/A US EQUITY ...,0.397072,Date PX CUR_MKT_CAP EQY_SH_OUT ...,12.481364,17.87,6.382543,12.481364,18.863907


In [23]:
slicer = dfutils.df_slicer()
alpha_ticker = 'WSC US EQUITY'
fperiod = "1BF"
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD yield', 'FCF yield']
unaffected_date = '2018-06-19'
unaff_dt = datetime.datetime.strptime(unaffected_date, '%Y-%m-%d')
tgt_dt = "2019-01-18"
price_tgt_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
as_of_dt = datetime.date.today()
analyst_upside = 15.5

In [25]:
peer_ticker_list = bloomberg_peers_from_r(alpha_ticker)
peer_ticker_list


['HRI US EQUITY',
 'URI US EQUITY',
 'HEES US EQUITY',
 'UHAL US EQUITY',
 'TGH US EQUITY',
 'MGRC US EQUITY']

In [26]:
peer2historical_mult_df = {p:multiples_df(p,slicer.prev_n_business_days(120,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod, multiples_to_query=metrics) for p in peer_ticker_list}
peer2historical_mult_df

{'HRI US EQUITY':            Date     PX  FCF yield      P/E  EV/EBITDA  EV/Sales  DVD yield
 0    2017-12-26  62.99  -0.062534  123.041      6.439     2.210        NaN
 1    2017-12-27  63.26  -0.062280  121.663      6.449     2.214        NaN
 2    2017-12-28  63.56  -0.061999  120.383      6.461     2.218        NaN
 3    2017-12-29  62.61  -0.062954  116.810      6.270     2.153        NaN
 4    2018-01-02  63.36  -0.061896  116.856      6.302     2.164        NaN
 5    2018-01-03  62.96  -0.062133  115.457      6.282     2.158        NaN
 6    2018-01-04  62.75  -0.062183  114.421      6.270     2.154        NaN
 7    2018-01-05  62.91  -0.062023  114.703      6.277     2.157        NaN
 8    2018-01-08  62.25  -0.062145  112.239      6.239     2.146        NaN
 9    2018-01-09  62.87  -0.062916   95.453      6.297     2.155        NaN
 10   2018-01-10  62.70  -0.062916   94.770      6.287     2.152        NaN
 11   2018-01-11  63.77  -0.061692   95.959      6.334     2.169       

In [27]:
alpha_historical_mult_df = multiples_df(alpha_ticker, slicer.prev_n_business_days(120,unaff_dt).strftime('%Y%m%d'), unaff_dt.strftime('%Y%m%d'), api_host, fperiod) # ['CID','Date','Ticker','PX','P/E','EV/EBITDA','EV/Sales','DVD yield','FCF yield']
ticker2short_ticker = {p:p.split(' ')[0] for p in peer_ticker_list+[alpha_ticker]}
alpha_historical_mult_df

,Date,PX,FCF yield,P/E,EV/EBITDA,EV/Sales,DVD yield
0,2017-12-26,12.80,NaN,217.356,12.806,3.237,NaN
1,2017-12-27,12.90,NaN,217.356,12.806,3.237,NaN
2,2017-12-28,12.85,NaN,217.356,12.806,3.237,NaN
3,2017-12-29,12.70,NaN,217.356,12.806,3.237,NaN
4,2018-01-02,12.95,NaN,217.356,12.806,3.237,NaN
5,2018-01-03,12.90,NaN,217.356,12.806,3.237,NaN
6,2018-01-04,12.95,NaN,217.356,12.806,3.237,NaN
7,2018-01-05,12.90,NaN,217.356,12.806,3.237,NaN
8,2018-01-08,12.45,NaN,217.356,12.806,3.237,NaN
9,2018-01-09,12.65,NaN,217.356,12.806,3.237,NaN


In [28]:
peer2ptd_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
peer2now_multiple = {p:multiples_df(p,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',multiples_to_query=metrics) for p in peer_ticker_list}
alpha_balance_sheet_df_ptd = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')
alpha_balance_sheet_df_now = multiple_underlying_df(alpha_ticker,slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF')

In [29]:
#Runs the regression on all metrics and find the optimal peer group per metric which it then uses to calculate the downside
rows=[]
metric2peer2coeff = {m:{} for m in metrics}
peers_d = {}
for metric in metrics:
    if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
        m_df = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
        for p in peer2historical_mult_df:
            m_df = pd.merge(m_df,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
        peer_tickers = [p for p in peer_ticker_list if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
        peer_tickers = [p for p in peer_tickers if ~peer2ptd_multiple[p][metric].isnull().all() and ~peer2now_multiple[p][metric].isnull().all()]
        m_ols_df = m_df[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in peer_tickers]]
        formula = alpha_ticker.split(' ')[0] + '~.' #+ " + ".join([t.split(' ')[0] for t in peer_ticker_list])
        r_df = ro.pandas2ri.py2ri(m_ols_df)
        r_df = stats.na_omit(r_df)
        model = stats.lm(formula, data = r_df)
        optimal_model = stats.step(model, direction = "backward", trace = False)
        summary_results = base.summary(optimal_model)
        summary_results2 = ro.pandas2ri.ri2py(summary_results)
        optimal_peers = base.attr(optimal_model.rx2('terms'), "term.labels")
        optimal_peers2 = []
        for i in optimal_peers:
            for k in peer_tickers:
                if k.startswith(i + ' '):
                    optimal_peers2.append(k)
                elif i.startswith('X') and k.startswith(i[1:] + ' '):
                    optimal_peers2.append(k)
                elif '.' in i:
                    i = i.replace('.', '/')
                    if k.startswith(i + ' '):
                        optimal_peers2.append(k)
                #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
        print(optimal_peers2)
        peers_d[metric] = optimal_peers2
        optimal_model_coeff = optimal_model.rx2('coefficients')
        print(optimal_model_coeff)
        optimal_model_coeff_df = pd.DataFrame(columns=['Peer','Coefficient'],data=[(optimal_peers2[count],optimal_model_coeff[count + 1]) for count in range(0,len(optimal_model_coeff.names)-1)])
        optimal_model_coeff_df = optimal_model_coeff_df.append({'Peer': 'Intercept', 'Coefficient': optimal_model_coeff[0]}, ignore_index = True)
        print(optimal_model_coeff_df)
        peer2coeff = {p:optimal_model_coeff.rx2(p)[0] for p in optimal_model_coeff.names}
        del peer2coeff["(Intercept)"]
        count = 0
        for k in optimal_peers:
            peer2coeff[optimal_peers2[count]] = peer2coeff.pop(k)
            count = count + 1
        peer2coeff["(Intercept)"] = optimal_model_coeff[0]
        metric2peer2coeff[metric] = peer2coeff
        rows.append([metric, summary_results2, optimal_model_coeff_df]) #+[peer2coeff[p] for p in optimal_peers2]+[peer2coeff['(Intercept)']])
    else:
        continue

['HRI US EQUITY', 'URI US EQUITY', 'HEES US EQUITY', 'TGH US EQUITY', 'MGRC US EQUITY']
(Intercept)         HRI         URI        HEES         TGH        MGRC 

-441.781058    1.495809  -20.381665    5.185528    3.241026   24.652812 

             Peer  Coefficient
0   HRI US EQUITY     1.495809
1   URI US EQUITY   -20.381665
2  HEES US EQUITY     5.185528
3   TGH US EQUITY     3.241026
4  MGRC US EQUITY    24.652812
5       Intercept  -441.781058
['HRI US EQUITY', 'URI US EQUITY', 'HEES US EQUITY', 'TGH US EQUITY', 'MGRC US EQUITY']
(Intercept)         HRI         URI        HEES         TGH        MGRC 

  9.7846334  -1.5629436   1.4656716   0.9729050   0.2886043  -0.8167636 

             Peer  Coefficient
0   HRI US EQUITY    -1.562944
1   URI US EQUITY     1.465672
2  HEES US EQUITY     0.972905
3   TGH US EQUITY     0.288604
4  MGRC US EQUITY    -0.816764
5       Intercept     9.784633
['HRI US EQUITY', 'URI US EQUITY', 'HEES US EQUITY', 'UHAL US EQUITY', 'MGRC US EQUITY']
(Inte

In [30]:
peer_set = {k:set(v) for k, v in peers_d.items()}
overlap_peers = set.intersection(*peer_set.values())

#for i in overlap_peers:
    #if '/' in i:
        #i.replace('/', '.')
overlap_peers
##Manually pull the overlap peer group

{'HEES US EQUITY', 'HRI US EQUITY', 'MGRC US EQUITY', 'URI US EQUITY'}

In [31]:
df = pd.DataFrame(columns=['Metric', 'OLS HTML', 'Model Coefficients'] ,data=rows)
df['Peers Multiples DataFrame @ Price Target Date'] = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
df['Alpha Implied Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+metric2peer2coeff[m]['(Intercept)'])
df['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df)
df['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Price Target Date'])]

df['Peers Multiples DataFrame @ Now']  = df['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in peers_d[m]]))
df['Alpha Implied Multiple @ Now'] = df['Metric'].apply(lambda m: sum([metric2peer2coeff[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in peers_d[m]])+metric2peer2coeff[m]['(Intercept)'])
df['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df)
df['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df['Metric'],df['Alpha Implied Multiple @ Now'])]

df['Alpha Upside (analyst)'] = analyst_upside
df['Premium'] = (df['Alpha Upside (analyst)'].astype(float)-df['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

df['Alpha Downside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)
df['Alpha Upside (Adj,weighted)'] = df['Alpha Unaffected PX @ Now'].astype(float)+df['Premium'].astype(float)


In [32]:
df #The non-overlap dataframe results

,Metric,OLS HTML,Model Coefficients,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Peers Multiples DataFrame @ Now,Alpha Implied Multiple @ Now,Alpha Balance Sheet DataFrame @ Now,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Premium,"Alpha Downside (Adj,weighted)","Alpha Upside (Adj,weighted)"
0,P/E,"[[function (formula, data, subset, weights, na...",Peer Coefficient 0 HRI US EQUI...,Peer Multiple 0 HRI US EQUITY ...,-85.783508,Date PX CUR_MKT_CAP EQY_SH_OUT ...,-9.693536,Peer Multiple 0 HRI US EQUITY ...,-89.371941,Date PX CUR_MKT_CAP EQY_SH_OUT ...,-10.724633,15.5,25.193536,-10.724633,14.468903
1,EV/EBITDA,"[[function (formula, data, subset, weights, na...",Peer Coefficient 0 HRI US EQUI...,Peer Multiple 0 HRI US EQUITY ...,9.389743,Date PX CUR_MKT_CAP EQY_SH_OUT ...,15.401462,Peer Multiple 0 HRI US EQUITY ...,9.451241,Date PX CUR_MKT_CAP EQY_SH_OUT ...,15.699398,15.5,0.098538,15.699398,15.797937
2,EV/Sales,"[[function (formula, data, subset, weights, na...",Peer Coefficient 0 HRI US EQUI...,Peer Multiple 0 HRI US EQUITY ...,3.035250,Date PX CUR_MKT_CAP EQY_SH_OUT ...,15.332517,Peer Multiple 0 HRI US EQUITY ...,3.058584,Date PX CUR_MKT_CAP EQY_SH_OUT ...,15.605787,15.5,0.167483,15.605787,15.773269


In [24]:
df["Model Coefficients"][0]

,Peer,Coefficient
0,TDC US EQUITY,0.445818
1,PEGA US EQUITY,0.044175
2,FEYE US EQUITY,-0.000022
3,PCTY US EQUITY,0.063865
4,ACIW US EQUITY,-0.073976
5,MIME US EQUITY,0.034600
6,IMPV US EQUITY,-0.109543
7,MB US EQUITY,-0.009669
8,OSPN US EQUITY,0.173298
9,AKAM US EQUITY,0.427513


In [20]:
#Regression of the overlap group which is then used to calculate downside based on all metrics
#rows2=[]
#metric2peer2coeff2={m:{} for m in metrics}
#for metric in metrics:
 #   if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
  #      m_df2 = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
   #     for p in overlap_peers:
    #        m_df2 = pd.merge(m_df2,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
     #   overlap_peers2 = [p for p in overlap_peers if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
      #  m_ols_df2 = m_df2[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in overlap_peers2]]
       # m_ols_df2.columns = ['X' + str(col) for col in m_ols_df2.columns]
        #regress a vs. p1,p2,...,pn
       # formula2 = "X" + alpha_ticker.split(' ')[0] + ' ~ ' + " + ".join(['X' + t.split(' ')[0] for t in overlap_peers2])
       # ols_result2 = sm.ols(formula=formula2, data=m_ols_df2).fit()
       # peer2coeff2 = {}
       # for p in overlap_peers2:
        #    peer2coeff2[p] = ols_result2.params['X' + p.split(' ')[0]]
        #peer2coeff2['Intercept'] = ols_result2.params['Intercept']
        #print(peer2coeff2)
        #metric2peer2coeff2[metric] = peer2coeff2
        #rows2.append([metric,ols_result2.summary().as_html()]+[peer2coeff2[p] for p in overlap_peers2]+[peer2coeff2['Intercept']])
    #else:
     #   continue

In [33]:
#Regression of the overlap group which is then used to calculate downside based on all metrics
rows2=[]
metric2peer2coeff2 = {m:{} for m in metrics}
for metric in metrics:
    if len(alpha_historical_mult_df[~pd.isnull(alpha_historical_mult_df[metric])])>0:
        m_df2 = alpha_historical_mult_df[['Date',metric]].rename(columns={metric:ticker2short_ticker[alpha_ticker]})
        for p in overlap_peers:
            m_df2 = pd.merge(m_df2,peer2historical_mult_df[p][['Date',metric]],how='left',on='Date').rename(columns={metric:ticker2short_ticker[p]})
        overlap_peers = [p for p in overlap_peers if len(m_df[~pd.isnull(m_df[p.split(' ')[0]])])>0] # remove peers with all nulls
        m_ols_df2= m_df2[[alpha_ticker.split(' ')[0]]+[t.split(' ')[0] for t in overlap_peers]]
        #regress a vs. p1,p2,...,pn
        formula = alpha_ticker.split(' ')[0] + ' ~. ' #+ " + ".join([t.split(' ')[0] for t in overlap_peers])
        r_df2 = ro.pandas2ri.py2ri(m_ols_df2)
        r_df2 = stats.na_omit(r_df2)
        model2 = stats.lm(formula, data = r_df2)
        #ols_result = sm.ols(formula=formula, data=m_ols_df).fit()
        summary = base.summary(model2)
        summary = ro.pandas2ri.ri2py(summary)
        model_coeff = model2.rx2('coefficients')
        model_terms = base.attr(model2.rx2('terms'), "term.labels")
        overlap_peers2 = []
        for i in model_terms:
            for k in overlap_peers:
                if k.startswith(i + ' '):
                    overlap_peers2.append(k)
                elif i.startswith('X') and k.startswith(i[1:] + ' '):
                    overlap_peers2.append(k)
                elif '.' in i:
                    i = i.replace('.', '/')
                    if k.startswith(i + ' '):
                        overlap_peers2.append(k)
                #Add the "__ EQUITY" phrase back into the optimal peer list for Bloomberg search
        peer2coeff2 = {}
        for p in range(0,len(overlap_peers)):
            peer2coeff2[overlap_peers2[p]] = model_coeff[p+1]
        peer2coeff2['Intercept'] = model_coeff[0]
        metric2peer2coeff2[metric] = peer2coeff2
        rows2.append([metric,summary]+[peer2coeff2[p] for p in overlap_peers2]+[peer2coeff2['Intercept']])
    else:
        continue

In [34]:
df2 = pd.DataFrame(columns=['Metric','OLS HTML'] + overlap_peers2 + ['Intercept'], data=rows2)
df2['Peers Multiples DataFrame @ Price Target Date'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2ptd_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers2]))
df2['Alpha Implied Multiple @ Price Target Date'] = df2['Metric'].apply(lambda m: sum([metric2peer2coeff2[m][p]*peer2ptd_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers2])+metric2peer2coeff2[m]['Intercept'])
df2['Alpha Balance Sheet DataFrame @ Price Target Date'] =  [alpha_balance_sheet_df_ptd]*len(df2)
df2['Alpha Unaffected PX @ Price Target Date'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_ptd) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Price Target Date'])]

df2['Peers Multiples DataFrame @ Now'] = df2['Metric'].apply(lambda m: pd.DataFrame(columns=['Peer','Multiple'],data=[(p,peer2now_multiple[p][m].fillna(0).iloc[-1]) for p in overlap_peers2]))
df2['Alpha Implied Multiple @ Now'] = df2['Metric'].apply(lambda m: sum([metric2peer2coeff2[m][p]*peer2now_multiple[p][m].fillna(0).iloc[-1] for p in overlap_peers2])+metric2peer2coeff2[m]['Intercept'])
df2['Alpha Balance Sheet DataFrame @ Now'] =  [alpha_balance_sheet_df_now]*len(df2)
df2['Alpha Unaffected PX @ Now'] = [compute_implied_price_from_multiple(m,mult,alpha_balance_sheet_df_now) for (m,mult) in zip(df2['Metric'],df2['Alpha Implied Multiple @ Now'])]

df2['Alpha Upside (analyst)'] = analyst_upside
df2['Premium'] = (df2['Alpha Upside (analyst)'].astype(float)-df2['Alpha Unaffected PX @ Price Target Date'].astype(float)).fillna(0).apply(lambda x: max(x,0))

df2['Alpha Downside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)
df2['Alpha Upside (Adj)'] = df2['Alpha Unaffected PX @ Now'].astype(float)+df2['Premium'].astype(float)

In [35]:
df2 #resulting dataframe for the overlap group

,Metric,OLS HTML,URI US EQUITY,HEES US EQUITY,MGRC US EQUITY,HRI US EQUITY,Intercept,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Peers Multiples DataFrame @ Now,Alpha Implied Multiple @ Now,Alpha Balance Sheet DataFrame @ Now,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Premium,Alpha Downside (Adj),Alpha Upside (Adj)
0,P/E,"[[function (formula, data, subset, weights, na...",-25.815181,6.943161,23.645264,1.803786,-369.699522,Peer Multiple 0 URI US EQUITY ...,-67.523376,Date PX CUR_MKT_CAP EQY_SH_OUT ...,-7.630141,Peer Multiple 0 URI US EQUITY ...,-68.992791,Date PX CUR_MKT_CAP EQY_SH_OUT ...,-8.279135,15.5,23.130141,-8.279135,14.851007
1,EV/EBITDA,"[[function (formula, data, subset, weights, na...",1.566340,1.080226,-0.978413,-1.746991,13.512792,Peer Multiple 0 URI US EQUITY ...,9.732428,Date PX CUR_MKT_CAP EQY_SH_OUT ...,16.548131,Peer Multiple 0 URI US EQUITY ...,9.825768,Date PX CUR_MKT_CAP EQY_SH_OUT ...,16.956266,15.5,0.000000,16.956266,16.956266
2,EV/Sales,"[[function (formula, data, subset, weights, na...",0.497668,0.543318,-0.327780,-1.101358,3.911241,Peer Multiple 0 URI US EQUITY ...,3.153732,Date PX CUR_MKT_CAP EQY_SH_OUT ...,16.556286,Peer Multiple 0 URI US EQUITY ...,3.162760,Date PX CUR_MKT_CAP EQY_SH_OUT ...,16.682894,15.5,0.000000,16.682894,16.682894


In [206]:
def compare_multiples(y_mult_df, x_mult_df, mult_colname):
    if len(y_mult_df[~pd.isnull(y_mult_df[mult_colname])]) == 0: return pd.DataFrame(columns=['Date','Multiple Ratio'])
    if len(x_mult_df[~pd.isnull(x_mult_df[mult_colname])]) == 0: return pd.DataFrame(columns=['Date','Multiple Ratio'])
    df = pd.merge(x_mult_df[['Date',mult_colname]],y_mult_df[['Date',mult_colname]], how='right',on='Date')
    df['Multiple Ratio'] = df[mult_colname+'_y']/df[mult_colname+'_x']
    return df[['Date','Multiple Ratio']].sort_values(by='Date')

In [207]:
def calibration_data(alpha_ticker, OLS_results, metrics):
    peer_tickers = OLS_results['overlap_peers']
    len_peer = len(peer_tickers)
    weight = [(1/len_peer)]*len_peer
    peer2weight = dict(zip(peer_tickers, weight))
    
    alpha_historical_mult = OLS_results['alpha_historical_mult_df']
    peer2historical_mult = OLS_results['peer2historical_mult_df']

    metric2rel = {}
    metrics2 = []
    for metric in metrics:
        if len(alpha_historical_mult[~pd.isnull(alpha_historical_mult[metric])])>0:
            alpha_over_peers_df = pd.DataFrame()
            alpha_over_peers_df['Date'] = alpha_historical_mult['Date']
            alpha_over_peer_df_list = []
            tot_adj_weight = 0
            peers_included = []
            for (peer,weight) in peer2weight.items():
                peer_mult = peer2historical_mult[peer]
                alpha_over_peer_df = compare_multiples(alpha_historical_mult,peer_mult,metric)
                if len(alpha_over_peer_df) > 0:
                    tot_adj_weight += weight
                    peers_included.append(peer)
                    alpha_over_peers_df = pd.merge(alpha_over_peers_df,alpha_over_peer_df,how='left',on='Date').rename(columns={'Multiple Ratio':'vs. '+peer})
                    alpha_over_peer_df_list.append(alpha_over_peer_df[['Date','Multiple Ratio']].rename(columns={'Multiple Ratio': 'vs. ' + peer}))

            peer2adj_weight = {p:(peer2weight[p]/tot_adj_weight) for p in peers_included}
            for p in peer2adj_weight:
                alpha_over_peers_df['vs. '+p+'(weighted)'] = peer2adj_weight[p]*alpha_over_peers_df['vs. '+p]

            alpha_over_peers_df['vs. all peers'] = alpha_over_peers_df[['vs. '+p+'(weighted)' for p in peers_included]].sum(axis=1)

            mu=alpha_over_peers_df['vs. all peers'].mean()
            sigma=alpha_over_peers_df['vs. all peers'].std()

            metrics2.append(metric)
            
            metric2rel[metric] = {
                'Mu':mu,
                'Sigma':sigma,
                'Alpha vs. all peers, dataframe': alpha_over_peers_df,
                'Alpha vs. each peer, list':alpha_over_peer_df_list,
                'Peers adjusted weight': peer2adj_weight
            }
        else:
            continue

    return {
        'metrics': metrics2,
        'peer_tickers': peer_tickers,
        'metric2rel': metric2rel,
        'alpha_historical_mult_df': alpha_historical_mult,
        'peer2historical_mult_df': peer2historical_mult
    }

In [209]:
calib_results = calibration_data("TPCO US EQUITY", OLS_results, metrics)

In [210]:
def metric2implied_px(alpha_ticker,peer_tickers, dt, metrics, api_host, metric2stat_rel, fperiod='1BF'):
    slicer = dfutils.df_slicer()

    start_date_yyyymmdd = slicer.prev_n_business_days(100,dt).strftime('%Y%m%d')
    peer2mult_df = {p:multiples_df(p,start_date_yyyymmdd, dt.strftime('%Y%m%d'), api_host, fperiod, multiples_to_query=metrics) for p in peer_tickers}
    print(peer2mult_df)
    alpha_mult_underlying_df = multiple_underlying_df(alpha_ticker,start_date_yyyymmdd,dt.strftime('%Y%m%d'),api_host,fperiod)

    metric2data={m:{} for m in metrics}
    for metric in metrics:
        #alpha_mult = alpha_mult_df[metric].iloc[-1]
        #alpha_px = alpha_mult_df['PX'].iloc[-1]
        alpha_balance_sheet_df = alpha_mult_underlying_df[alpha_mult_underlying_df['Date']==dt.strftime('%Y-%m-%d')]
        peer2mult = {p:peer2mult_df[p][metric].iloc[-1] for p in peer_tickers}

        stat_rel = metric2stat_rel[metric]
        mu = stat_rel['Mu']
        sigma = stat_rel['Sigma']
        peer2adj_weight = stat_rel['Peers adjusted weight']
        #peers_multiple = sum([peer2adj_weight[p]*peer2mult[p] for p in peer_tickers])
        peers_multiple = sum([(peer2adj_weight[p]*peer2mult[p] if p in peer2adj_weight else 0.0)  for p in peer_tickers])

        implied_multiple_high = (mu+2*sigma)*peers_multiple
        implied_multiple_mean = mu*peers_multiple
        implied_multiple_low = (mu-2*sigma)*peers_multiple

        if metric in ['FCF yield','DVD yield']: # flip
            tmp = implied_multiple_high
            implied_multiple_high = implied_multiple_low
            implied_multiple_low = tmp

        metric2data[metric]['Alpha implied multiple (high)'] = implied_multiple_high
        metric2data[metric]['Alpha implied multiple (mean)'] = implied_multiple_mean
        metric2data[metric]['Alpha implied multiple (low)'] = implied_multiple_low
        metric2data[metric]['Alpha Balance Sheet DataFrame'] = alpha_balance_sheet_df
        #metric2data[metric]['Alpha observed multiple'] = alpha_mult ### takes extra api call, use only if needed
        metric2data[metric]['Peers multiple'] = peers_multiple
        metric2data[metric]['Peer2Multiple'] = peer2mult
        metric2data[metric]['Alpha Unaffected PX (-2sigma)'] = compute_implied_price_from_multiple(metric,implied_multiple_low,alpha_balance_sheet_df)
        metric2data[metric]['Alpha Unaffected PX (avg)'] = compute_implied_price_from_multiple(metric,implied_multiple_mean,alpha_balance_sheet_df)
        metric2data[metric]['Alpha Unaffected PX (+2sigma)'] = compute_implied_price_from_multiple(metric,implied_multiple_high,alpha_balance_sheet_df)
        #metric2data[metric]['Alpha PX'] = alpha_px

    return metric2data

In [211]:
def premium_analysis_df(alpha_ticker, tgt_dt, analyst_upside, calib_results, api_host):
    peers = calib_results['peer_tickers']
    last_price_target_dt = datetime.datetime.strptime(tgt_dt, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    metric2stat_rel = calib_results['metric2rel']
    metrics = calib_results['metrics']
    
    metric2implied_now = metric2implied_px(alpha_ticker,peers,as_of_dt,metrics,api_host,metric2stat_rel,fperiod='1BF')
    metric2implied_at_price_tgt_date = metric2implied_px(alpha_ticker,peers,last_price_target_dt,metrics,api_host,metric2stat_rel,fperiod='1BF')
    print(metric2implied_at_price_tgt_date)
    
    df = pd.DataFrame()
    df['Metric'] = metrics
    df['Alpha to Peer historical ratio (mean)'] = df['Metric'].apply(lambda m: metric2stat_rel[m]['Mu'])
    df['Alpha to Peer historical ratio (std)'] = df['Metric'].apply(lambda m: metric2stat_rel[m]['Sigma'])

    df['Peers Composite Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: metric2implied_at_price_tgt_date[m]['Peers multiple'])
    df['Peer2Multiple @ Price Target Date'] = df['Metric'].apply(lambda m: metric2implied_at_price_tgt_date[m]['Peer2Multiple'])
    df['Alpha Implied Multiple (mean) @ Price Target Date'] = df['Metric'].apply(lambda m: metric2implied_at_price_tgt_date[m]['Alpha implied multiple (mean)'])
    df['Alpha Balance Sheet @ Price Target Date'] = df['Metric'].apply(lambda m: metric2implied_at_price_tgt_date[m]['Alpha Balance Sheet DataFrame'])
    df['Alpha Unaffected PX @ Price Target Date'] = df['Metric'].apply(lambda m: metric2implied_at_price_tgt_date[m]['Alpha Unaffected PX (avg)'])
    df['Premium'] = None
    df['Alpha Upside (analyst)'] = analyst_upside
    df['Premium'] = df['Alpha Upside (analyst)'] - df['Alpha Unaffected PX @ Price Target Date']


    df['Peers Composite Multiple @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Peers multiple'])
    df['Peer2Multiple @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Peer2Multiple'])
    df['Alpha Implied Multiple (mean) @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha implied multiple (mean)'])
    df['Alpha Balance Sheet @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha Balance Sheet DataFrame'])
    df['Alpha Unaffected PX (low) @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha Unaffected PX (-2sigma)'])
    df['Alpha Unaffected PX (mean) @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha Unaffected PX (avg)'])
    df['Alpha Unaffected PX (high) @ Now'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha Unaffected PX (+2sigma)'])


    df['Alpha Downside (Adj)'] = df['Alpha Unaffected PX (low) @ Now']
    df['Alpha Upside (Adj)'] = df['Alpha Unaffected PX (mean) @ Now'] + df['Premium']
    #df['Alpha Downside (Adj,weighted)'] = df['Alpha Downside (Adj)'].astype(float)
    #df['Alpha Upside (Adj,weighted)'] = df['Alpha Upside (Adj)'].astype(float)

    #more potential items
    #df['Alpha Current Multiple'] = df['Metric'].apply(lambda m: metric2implied_now[m]['Alpha observed multiple'])

    return df

In [212]:
premium_analysis_df("TPCO US EQUITY", '2018-12-03', 19, calib_results, api_host)

{'JW/A US EQUITY':            Date     PX     P/E  EV/EBITDA  EV/Sales  DVD yield
0    2018-07-13  66.40  20.627      9.970     2.242        NaN
1    2018-07-16  67.45  20.940     10.110     2.274        NaN
2    2018-07-17  68.25  21.181     10.235     2.302        NaN
3    2018-07-18  67.60  20.972     10.155     2.285        NaN
4    2018-07-19  66.70  20.686     10.044     2.260        NaN
5    2018-07-20  66.00  20.467      9.906     2.229        NaN
6    2018-07-23  64.40  19.959      9.679     2.179        NaN
7    2018-07-24  63.75  19.750      9.588     2.159        NaN
8    2018-07-25  63.55  19.682      9.555     2.152        NaN
9    2018-07-26  63.50  19.659      9.538     2.148        NaN
10   2018-07-27  62.25  19.271      9.364     2.109        NaN
11   2018-07-30  62.05  19.197      9.330     2.102        NaN
12   2018-07-31  63.15  19.531      9.980     2.249        NaN
13   2018-08-01  62.15  19.215      9.852     2.220        NaN
14   2018-08-02  62.65  19.363      

{'JW/A US EQUITY':            Date     PX     P/E  EV/EBITDA  EV/Sales  DVD yield
0    2018-07-12  65.90  20.473      9.895     2.225        NaN
1    2018-07-13  66.40  20.627      9.970     2.242        NaN
2    2018-07-16  67.45  20.940     10.110     2.274        NaN
3    2018-07-17  68.25  21.181     10.235     2.302        NaN
4    2018-07-18  67.60  20.972     10.155     2.285        NaN
5    2018-07-19  66.70  20.686     10.044     2.260        NaN
6    2018-07-20  66.00  20.467      9.906     2.229        NaN
7    2018-07-23  64.40  19.959      9.679     2.179        NaN
8    2018-07-24  63.75  19.750      9.588     2.159        NaN
9    2018-07-25  63.55  19.682      9.555     2.152        NaN
10   2018-07-26  63.50  19.659      9.538     2.148        NaN
11   2018-07-27  62.25  19.271      9.364     2.109        NaN
12   2018-07-30  62.05  19.197      9.330     2.102        NaN
13   2018-07-31  63.15  19.531      9.980     2.249        NaN
14   2018-08-01  62.15  19.215      

{'P/E': {'Alpha implied multiple (high)': 26.26178432724754, 'Alpha implied multiple (mean)': 21.350581190853376, 'Alpha implied multiple (low)': 16.439378054459212, 'Alpha Balance Sheet DataFrame':         Date     PX  CUR_MKT_CAP  EQY_SH_OUT  BEST_EBITDA  BEST_SALES  \
0 2018-12-03  14.84     528.3242      35.601        99.98     990.753   

   BEST_EPS  DIVIDEND_INDICATED_YIELD  BEST_OPP  BEST_NET_INCOME  BEST_CAPEX  \
0     0.155                       NaN     24.65           18.654         NaN   

   CUR_EV_COMPONENT  
0           -51.033  , 'Peers multiple': 13.545666666666666, 'Peer2Multiple': {'JW/A US EQUITY': 16.863, 'GCI US EQUITY': 10.255, 'GTN US EQUITY': 13.519}, 'Alpha Unaffected PX (-2sigma)': 2.5481035984411777, 'Alpha Unaffected PX (avg)': 3.3093400845822734, 'Alpha Unaffected PX (+2sigma)': 4.070576570723369}, 'EV/EBITDA': {'Alpha implied multiple (high)': 13.981777534121509, 'Alpha implied multiple (mean)': 8.739217916275177, 'Alpha implied multiple (low)': 3.4966582

,Metric,Alpha to Peer historical ratio (mean),Alpha to Peer historical ratio (std),Peers Composite Multiple @ Price Target Date,Peer2Multiple @ Price Target Date,Alpha Implied Multiple (mean) @ Price Target Date,Alpha Balance Sheet @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Premium,Alpha Upside (analyst),Peers Composite Multiple @ Now,Peer2Multiple @ Now,Alpha Implied Multiple (mean) @ Now,Alpha Balance Sheet @ Now,Alpha Unaffected PX (low) @ Now,Alpha Unaffected PX (mean) @ Now,Alpha Unaffected PX (high) @ Now,Alpha Downside (Adj),Alpha Upside (Adj)
0,P/E,1.576193,0.181283,13.545667,"{'JW/A US EQUITY': 16.863, 'GCI US EQUITY': 10...",21.350581,Date PX CUR_MKT_CAP EQY_SH_OUT ...,3.309340,15.690660,19,13.314333,"{'JW/A US EQUITY': 16.718, 'GCI US EQUITY': 10...",20.985955,Date PX CUR_MKT_CAP EQY_SH_OUT ...,2.520746,3.273809,4.026872,2.520746,18.964469
1,EV/EBITDA,1.221585,0.366408,7.154000,"{'JW/A US EQUITY': 8.791, 'GCI US EQUITY': 4.2...",8.739218,Date PX CUR_MKT_CAP EQY_SH_OUT ...,25.976237,-6.976237,19,7.075000,"{'JW/A US EQUITY': 8.739, 'GCI US EQUITY': 4.2...",8.642713,Date PX CUR_MKT_CAP EQY_SH_OUT ...,11.143701,25.702303,40.260906,11.143701,18.726067
2,EV/Sales,0.644018,0.146291,1.764000,"{'JW/A US EQUITY': 1.932, 'GCI US EQUITY': 0.4...",1.136047,Date PX CUR_MKT_CAP EQY_SH_OUT ...,33.048937,-14.048937,19,1.741000,"{'JW/A US EQUITY': 1.921, 'GCI US EQUITY': 0.4...",1.121235,Date PX CUR_MKT_CAP EQY_SH_OUT ...,18.457208,32.630009,46.802809,18.457208,18.581072
